In [25]:
import sys
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [26]:
from himpy.histogram import operations, expressionOperations, Histogram1D
from himpy.executor import Parser, Evaluator
from himpy.utils import E

In [27]:
%load_ext autoreload
%autoreload 2

# dataset generator
from utils.datasets import ColorImageGenerator

# feature extraction
from utils.feature_extraction import (
    FeatureMerger,
    ColorSetTransformer,
    create_histogram,
    create_histogram_
)

# search engine
from utils.search_engine import SearchEngine

# plot
from utils.plot.plotly_plot import (
    show_color_elements,
    show_complete_histogram,
    show_rank_images
)

# image colors
from utils.feature_extraction.color import COLOR_ELEMENTS, COLOR_ELEMENTS_RGB

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
image_generator = ColorImageGenerator()
color_transformer = ColorSetTransformer()
parser = Parser()

In [29]:
# Definition of high-level positional elements

Ec_green        = E("e1+e2+e3+e4+e5+e6+e7+e8+e9+e10+e11+e12+e13+e14+e15+e16+e17+e18+e19+e20")
Ec_yellow_green = E("e2+e3+e21+e22+e23+e24+e25+e26+e27+e28+e29+e30")
Ec_red          = E("e31+e32+e33+e34+e35+e36+e37+e38+e39+e40")
Ec_rose         = E("e32+e35+e36+e39+e40")


Ecs = [
    ("green", Ec_green),
    ("yellow_green", Ec_yellow_green),
    ("red", Ec_red),
    ("rose", Ec_rose)
]


# Sets of hight-level color elements (they will be used for the Evaluator below)
Ecs_set = { name: parser.parse_set(Ec.value) for name, Ec in Ecs}
high_level_elements = Ecs_set
high_level_elements

{'green': {'e1',
  'e10',
  'e11',
  'e12',
  'e13',
  'e14',
  'e15',
  'e16',
  'e17',
  'e18',
  'e19',
  'e2',
  'e20',
  'e3',
  'e4',
  'e5',
  'e6',
  'e7',
  'e8',
  'e9'},
 'yellow_green': {'e2',
  'e21',
  'e22',
  'e23',
  'e24',
  'e25',
  'e26',
  'e27',
  'e28',
  'e29',
  'e3',
  'e30'},
 'red': {'e31', 'e32', 'e33', 'e34', 'e35', 'e36', 'e37', 'e38', 'e39', 'e40'},
 'rose': {'e32', 'e35', 'e36', 'e39', 'e40'}}

## Image Retrieval

In [30]:
# Images with normal distrubited some elements 
# images = [
#     image_generator.generate(
#         shape=(100, 100), 
#         steps=(20, 20)) 
#     for i in range(10)
# ]

In [31]:
# Create histograms for the images
# hists = list()
# limit = len(images)

# for indx, image in enumerate(images):
#     color_image = color_transformer.fit_transform(X=image, y=None)
#     hist = create_histogram((color_image,))
#     hists.append((indx, hist))
#     print("\rCurrent image index: {}/{}".format(indx + 1, limit), end="")

with open("data.txt", "r") as data_file:
    line = data_file.readline().strip()
    id = 0
    element = ""
    frequency = 0.0
    hist = {}
    is_new_hist = True
    is_element_key = True
    hists_pairs = []

    while line:
        if is_new_hist:
            id = int(line)
            is_new_hist = False
        elif line == "X":
            hists_pairs.append((id, hist))
            hist = {}
            is_new_hist = True
        elif is_element_key:
            element = line
            is_element_key = False
        else:
            frequency = float(line)
            hist[element] = frequency
            is_element_key = True
        
        line = data_file.readline().strip()

print(hists_pairs[:2])
hists = [(id, Histogram1D(hist)) for id, hist in hists_pairs]
print(hists[:2])

[(0, {'e1': 0.04, 'e10': 0.04, 'e13': 0.04, 'e15': 0.04, 'e17': 0.08, 'e18': 0.04, 'e2': 0.12, 'e24': 0.04, 'e25': 0.04, 'e29': 0.04, 'e30': 0.04, 'e32': 0.04, 'e33': 0.04, 'e38': 0.08, 'e4': 0.04, 'e7': 0.12, 'e9': 0.12}), (1, {'e1': 0.08, 'e11': 0.04, 'e14': 0.04, 'e15': 0.04, 'e16': 0.04, 'e17': 0.04, 'e19': 0.04, 'e2': 0.04, 'e20': 0.04, 'e21': 0.04, 'e24': 0.04, 'e29': 0.16, 'e31': 0.04, 'e32': 0.04, 'e35': 0.04, 'e36': 0.04, 'e38': 0.04, 'e39': 0.04, 'e7': 0.04, 'e8': 0.08})]
[(0, <himpy.histogram.Histogram1D object at 0x00000172B22F9720>), (1, <himpy.histogram.Histogram1D object at 0x00000172B22F9A50>)]


In [32]:
import pickle

# with open("hists.pkcl", "wb") as f:
#     pickle.dump(hists, f)
# with open("images.pkcl", "wb") as f:
#     pickle.dump(images, f)
    
# with open("hists1.txt", "w") as file:
#     for id, hist in hists:
#         file.write(str(id) + "\n" + str(hist) + "X\n")

In [33]:
# Initialize a search engine
evaluator = Evaluator(operations, expressionOperations, high_level_elements=high_level_elements)
search_engine_default = SearchEngine(hists, parser, evaluator, use_index=False)
search_engine_inverted_index = SearchEngine(hists, parser, evaluator, use_index=True)

{}


In [34]:
TOP_N = 20

In [35]:
# Elements
E1 = E("green")
E2 = E("yellow_green")
E3 = E("red")
E4 = E("rose")
E5 = E("any")

# Define your query
query = E1 + E3

# Retrieve images using the query
ranked_images = search_engine_inverted_index.retrieve(query, top_n=100)
print("Total retrieved images:", len(ranked_images))
# with open("result.txt", "w") as file:
#     file.write(str(ranked_images))
# ranked_images

Total retrieved images: 0


In [36]:
fig = show_rank_images(images, ranked_images, limit=TOP_N, 
                       title="Top {}: <b>{}</b>".format(TOP_N, query.value))
fig.show()

NameError: name 'images' is not defined

In [ ]:
for id, score in ranked_images:
    green = evaluator.eval(["green"], hists[id][1]).sum() * 25
    yellow_green = evaluator.eval(["yellow_green"], hists[id][1]).sum() * 25
    red = evaluator.eval(["red"], hists[id][1]).sum() * 25
    rose = evaluator.eval(["rose"], hists[id][1]).sum() * 25
    print(f"id: {id:>2}, green: {round(green):>2}, yellow: {round(yellow_green):>2}, red: {round(red):>2}, rose: {round(rose):>2}")

In [ ]:
ranked_images2 = search_engine_default.retrieve(query, top_n=TOP_N)
print("Total retrieved images:", len(ranked_images2))
ranked_images2[:5]

In [ ]:
fig = show_rank_images(images, ranked_images2, limit=TOP_N, 
                       title="Top {}: <b>{}</b>".format(TOP_N, query.value))
fig.show()